In [17]:
from gurobipy import quicksum
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

model = gp.Model("timetable_discrete")

machines = 3
timeline = 48
ordres = [
    {'numéro': 1, 'duration': 2, 'due date': 10}, {'numéro': 2, 'duration': 2, 'due date': 10},
    {'numéro': 3, 'duration': 1, 'due date': 10}, {'numéro': 4, 'duration': 2, 'due date': 10},
    {'numéro': 5, 'duration': 3, 'due date': 15}, {'numéro': 6, 'duration': 4, 'due date': 15},
    {'numéro': 7, 'duration': 2, 'due date': 20}, {'numéro': 8, 'duration': 4, 'due date': 20},
    {'numéro': 9, 'duration': 2, 'due date': 30}, {'numéro': 10, 'duration': 2, 'due date': 30},
    {'numéro': 11, 'duration': 2, 'due date': 30}, {'numéro': 12, 'duration': 2, 'due date': 30},
    {'numéro': 13, 'duration': 10, 'due date': 40}, {'numéro': 14, 'duration': 2, 'due date': 45},
    {'numéro': 15, 'duration': 8, 'due date': 40}, {'numéro': 16, 'duration': 5, 'due date': 40},
    {'numéro': 17, 'duration': 2, 'due date': 35}, {'numéro': 18, 'duration': 12, 'due date': 30},
    {'numéro': 19, 'duration': 2, 'due date': 30}, {'numéro': 20, 'duration': 3, 'due date': 30}
]
#CHANGEMENT POUR AJOUTER LA CONTRAINTE DE CHANGEMENT 
T_change = 1
change = model.addVars(machines, timeline, vtype=GRB.BINARY, name="change")
#//// CHANGEMENT POUR AJOUTER LA CONTRAINTE DE CHANGEMENT 

x = {}
for i, ordre in enumerate(ordres):
    for m in range(machines):
        for t in range(timeline):  # tmax = timeline - durée + 1
            x[i, t, m] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_t{t}_m{m}")

model.setObjective(0, GRB.MINIMIZE)

for i, ordre in enumerate(ordres):
    model.addConstr(
        gp.quicksum(x[i, t, m] for m in range(machines)
                    for t in range(ordre['due date'])) == ordre['duration'],name=f"assign_{i}")

for m in range(machines):
    for t in range(timeline):
        model.addConstr(
            gp.quicksum(
                x[i, t, m]
                for i, ordre in enumerate(ordres) )<= 1,
            name=f"machine_{m}time{t}")
        
#//// CHANGEMENT POUR AJOUTER LA CONTRAINTE DE CHANGEMENT 

#Prendre en compte un changement 
for m in range(machines):
    for t in range(timeline - 1):
        for i in range(len(ordres)):
            # Si une tâche i commence à t+1 sur machine m, et n'était pas là à t
            model.addConstr(change[m, t] == ( x[i, t, m]-x[i, t + 1, m]),
                            name=f"start_detect_i{i}_m{m}_t{t}")
            
#Mettre la pause dans l'emploi du temps


for m in range(machines):
    for t in range(timeline):
        if t + T_change < timeline:
            for k in range(1, T_change + 1):
                if change[m, t] == 1:
                    for i in range(len(ordres)):
                        
                        model.addConstr(
                            x[i, m, t + k] <= (1-change[m,t]),
                            name=f"Constraint_{m}_{t}_{k}"
                        )


#//// CHANGEMENT POUR AJOUTER LA CONTRAINTE DE CHANGEMENT 

model.optimize()

# x = {(i, t, m): var}  avec var.X > 0.5 si l’ordre i commence à t sur machine m
# ordres = {i: {'numéro': id, 'duration': d}}

schedule = []

if model.status == GRB.OPTIMAL:
    for (i, t, m), var in x.items():
        if var.X > 0.5:
            duration = ordres[i]['duration']
            order_id = ordres[i]['numéro']
            schedule.append({
                'Machine': m,
                'Start': t,
                'Duration': duration,
                'Order': f"Ordre {order_id}"
            })
else:
    print("Pas de solution optimale trouvée.")
        

# Regrouper par machine
machines = sorted(set(task['Machine'] for task in schedule))
machine_to_y = {m: idx for idx, m in enumerate(machines)}

# Tracer
fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.tab20.colors  # palette de couleurs

for idx, task in enumerate(schedule):
    y = machine_to_y[task['Machine']]
    ax.barh(y, task['Duration'], left=task['Start'], height=0.4,
            color=colors[idx % len(colors)], edgecolor='black')
    ax.text(task['Start'] + task['Duration']/2, y, task['Order'],
            va='center', ha='center', fontsize=9, color='white')

# Axe y avec noms de machines
ax.set_yticks(range(len(machines)))
ax.set_yticklabels([f"Machine {m}" for m in machines])
ax.set_xlabel("Temps")
ax.set_title("Emploi du temps des ordres par machine")
ax.grid(True)

plt.tight_layout()
plt.show()

GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)

In [ ]:
#//// CHANGEMENT POUR AJOUTER LA CONTRAINTE DE CHANGEMENT 
#Prendre en compte un changement 
for i, ordre in enumerate(ordres):
    for m in range(machines):
        for t in range(timeline):
            model.addConstr(
                (x[i,m,t] != x[i, m, t + 1]) >> (change[m, t] == 1)  #les >> c'est pour faire du si ... alors 
            )
            model.addConstr(
                (x[i,m, t] == x[i,m, t + 1]) >> (change[m, t] == 0)
            )
#Mettre la pause dans l'emploi du temps


for m in machines:
    for t in range(len(timeline)):
        if t + T_change < max(T):
            for k in range(1, T_change + 1):
                if change[m, t] == 1:
                    model.addConstr(
                        gp.quicksum(x[i, m, t + k] for i in range(len(ordres))) == 0,
                        name=f"Constraint_{m}_{t}_{k}"
                    )

In [16]:
ordres = [
    {'numéro': 1, 'duration': 2, 'due date': 10}, {'numéro': 2, 'duration': 2, 'due date': 10},
    {'numéro': 3, 'duration': 1, 'due date': 10}, {'numéro': 4, 'duration': 2, 'due date': 10},
    {'numéro': 5, 'duration': 3, 'due date': 15}, {'numéro': 6, 'duration': 4, 'due date': 15},
    {'numéro': 7, 'duration': 2, 'due date': 20}, {'numéro': 8, 'duration': 4, 'due date': 20},
    {'numéro': 9, 'duration': 2, 'due date': 30}, {'numéro': 10, 'duration': 2, 'due date': 30},
    {'numéro': 11, 'duration': 2, 'due date': 30}, {'numéro': 12, 'duration': 2, 'due date': 30},
    {'numéro': 13, 'duration': 10, 'due date': 40}, {'numéro': 14, 'duration': 2, 'due date': 45},
    {'numéro': 15, 'duration': 8, 'due date': 40}, {'numéro': 16, 'duration': 5, 'due date': 40},
    {'numéro': 17, 'duration': 2, 'due date': 35}, {'numéro': 18, 'duration': 12, 'due date': 30},
    {'numéro': 19, 'duration': 2, 'due date': 30}, {'numéro': 20, 'duration': 3, 'due date': 30}
]

print(len(ordres))


20
